In [1]:
import geopandas as gpd

import fiona
import rasterio
from rasterio import features
from rasterio.features import shapes
from shapely.geometry import mapping, shape
from osgeo import gdal, gdalnumeric, ogr, osr
from gdalconst import *
from PIL import Image, ImageDraw

import os
import shutil
import numpy as np
import subprocess

%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from library.geoprocess import rm_and_mkdir, shp_to_shps, raster_to_rasters, polygonize

#### Create individual shapefiles of each country from shapefile of all countries
* load shapefile of all admin areas / countries as geodataframe
* filter out countries not internationally recognized
* loop through rows of geodataframe and save each row as a country-specific shapefile

In [2]:
# load shapefile of all admin areas / countries as geodataframe
gdf = gpd.read_file('data/geo/countries/countries_nf2.shp'); gdf.head(3)

# filter out countries not internationally recognized
country_filter1 = gdf['WB_A3'] != '-99'
gdf = gdf.drop_duplicates(subset='WB_A3')
gdf = gdf[country_filter1].set_index('WB_A3')

# loop through rows of geodataframe and save each row as a country-specific shapefile in newly created dir
# shp_to_shps('data/geo/countries/shp', gdf)

/Users/peter/anaconda/lib/python2.7/site-packages/geopandas/geodataframe.py:376: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


#### Generate city boundaries
* Clip master raster from 2013 by each country shapefile, creating country-specific rasters
* Use subprocess module to run gdal commands in terminal to do this
* Polygonize each country raster
* Select subset of polygons that have light intensity greater than selected thresh
* Union remaining polygons to get contiguous city boundaries
* Intersect with populated places to eliminate non-key cities
* Save outputs to cities directory

In [3]:
# clip master raster from 2013 by each country shapefile to create country-level rasters
input_tif_path = 'data/geo/images/F182013.v4c_web.stable_lights.avg_vis.tif'
input_shp_dir = 'data/geo/countries/shp'
output_tif_dir = 'data/geo/countries/tif'
countries = [x.encode('UTF-8') for x in gdf.index.values]
# raster_to_rasters(countries, input_tif_path, input_shp_dir, output_tif_dir)

In [4]:
# polygonize rasters and save to target directory
input_tif_dir = 'data/geo/countries/tif'
output_shp_dir = 'data/geo/countries/poly'
# polygonize(input_tif_dir, output_shp_dir, countries)

In [50]:
# Loop through each shapefile, filter out values less than 20, and union and dissolve the rest
country = 'AGO'
input_country_filename = country + '.shp'
input_shp_dir = 'data/geo/countries/poly'
input_shp_path = os.path.join(input_shp_dir, input_country_filename)
gdf_country = gpd.read_file(input_shp_path)
gdf_country.rename(columns={'country': 'val', 'val': 'country'}, inplace=True)
thresh = 25
gdf_country = gdf_country[gdf_country['val'] >= thresh]
polys = gdf_country.geometry
poly = polys.unary_union; type(poly)
poly_country = [country, poly]; poly_country
gdf_poly_country = gpd.GeoDataFrame(poly_country).T.rename(columns={0: 'country', 1: 'geometry'}); gdf_poly_country

,country,geometry
0,AGO,(POLYGON ((15.91128450449845 -17.3814745598568...


In [53]:
gdf_poly_country.crs = {'init': 'epsg:4326', 'no_defs': True}

# make temp folder to hold multi-polygon unioned shapes
# ADD

# write multi-polygon shapefile to temp foldf
gdf_poly_country.to_file('test.shp')

# split multi-polygons into polygons and write to new folder

# create new folder 
# open the original MultiPolygon file
with fiona.open('test.shp') as input:
    # create the new file: the driver, crs and schema are the same
    with fiona.open('output.shp','w',driver=input.driver, crs=input.crs, schema=input.schema) as output:
        # read the input file
        for multi in input:
            # extract each Polygon feature
            for poly in shape(multi['geometry']):
                # write the Polygon feature
                output.write({'properties': multi['properties'],'geometry': mapping(poly)})

In [36]:
from osgeo import ogr
from shapely.geometry import Polygon

# Here's an example Shapely geometry
#poly = Polygon([(0, 0), (0, 1), (1, 1), (0, 0)])

# Now convert it to a shapefile with OGR    
driver = ogr.GetDriverByName('Esri Shapefile')
ds = driver.CreateDataSource('my.shp')
layer = ds.CreateLayer('', None, ogr.wkbPolygon)
# Add one attribute
layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
defn = layer.GetLayerDefn()

## If there are multiple geometries, put the "for" loop here

# Create a new feature (attribute and geometry)
feat = ogr.Feature(defn)
feat.SetField('id', 123)

# Make a geometry, from Shapely object
geom = ogr.CreateGeometryFromWkb(poly.wkb)
feat.SetGeometry(geom)

layer.CreateFeature(feat)
feat = geom = None  # destroy these

# Save and close everything
ds = layer = feat = geom = None


AttributeError: 'NoneType' object has no attribute 'CreateLayer'

In [ ]:
pixel_vals = eth[0].flatten()  # flatten pixels

def filter_nodata(vals, no_data_val):  # filter function to remove zeros and nodata values from numpified raster values
    return vals != no_data_val
bool_arr_1 = np.array([filter_nodata(val, 255) for val in pixel_vals])
pixel_vals_nd_1 = pixel_vals[bool_arr_1]

bool_arr_2 = np.array([filter_nodata(val, 0) for val in pixel_vals_nd_1]) 
pixel_vals_nd_2 = pixel_vals_nd_1[bool_arr_2]

In [ ]:
# see what resulting histogram of data looks like
n, bins, patches = plt.hist(pixel_vals_nd_2, 50, normed=1, facecolor='green', alpha=0.75)

In [ ]:
from scipy.stats import expon
import math

In [ ]:
np.mean(pixel_vals_nd_2)

In [ ]:
# fit exponential curve to data, inspect raster data in qgis
# see what percent of non-zero pixels constitute cities at given cutoff threshold
params = expon.fit(pixel_vals_nd_2)
the_mean = params[1]
the_lambda = 1 / the_mean
thresh = 25
1 - math.exp(-thresh*the_lambda)